## Introduction

This notebook walks through the discovery, download, and plotting of remote sensing ocean SST level 2 (L2) data over a geographic region of interest defined by the user (through the use of common GIS files such as .shp) during the 2020 hurricane season. 

### Explore Gulf of Mexico SSTs during the 2020 hurricane season 

The evolution of sea surface temperature (SST) anomalies in the Gulf of Mexico before, during, and after Tropical Cyclones can be explored using NASA remote sensing data, such as the MODIS Aqua SST dataset. An ocean response can often be seen in the wake of hurricane tracks, with cold wakes, or areas of cooler water, along or to the right of hurricane tracks in association with wind-induced water column mixing that brings cooler waters at depth to the surface. These cold signatures are generally patchy and spatially confined. 

As NASA Earthdata transitions to the Earthdata Cloud, it will be common for some data to exist in a traditional on-premise storage system, accessed by direct download to a local environment, while other data will have migrated to the cloud system. This need to access and customize data from the Earthdata Cloud, and work with it locally is described in this notebook. This access workflow of cloud-based data and downloding to local environment for further analysis or use is presented in the Introduction (Part I) of the workshop, and described again here:

<img align="center"
     src="shpSearch_downloadLocal_diagram.png">

<img align="center"
     src="tutorial1_data_MODIS_browse.png">

*Note: In this example we are using a shape file to search on ocean remote sensing data, although one can do this type of search on any Earthdata (DAAC) data. For example, of particular interest could be using shape files to search on terrestrial hydrology data (e.g watersheds), globally (depending on data availability of course).* 

*Note: Here we show a programmatic way of completing this type of workflow, but this can also be accomplished through the Earthdata Search user interface. For a video tutorial on how to do this, please see https://www.youtube.com/watch?v=d1BR8w3u0dI&list=PLDWiCz1Ka4kSbqkoeOcPXGAv0gp8OS1Ah&index=7, and disregard the comments about 'UAT' (test environment), as this capability has since become operational. Use https://search.earthdata.nasa.gov/ and follow the rest of the steps in the video tutorial. Search by HUC (hydrologic unit code) capability is also available, if you are working with data over the United States. A video tutorial on how to search by HUC in Earthdata Search is available here: https://www.youtube.com/watch?v=8TLJOFe7XPw&list=PLDWiCz1Ka4kSbqkoeOcPXGAv0gp8OS1Ah&index=6*

#### Learning Objectives:
0. Earthdata Login Authentication (for download access from Earthdata data archive)
1. Search CMR for collection and granule IDs, using the collection shortname and provider
2. Download a file from the PO.DAAC (Earthdata) cloud archive to local computer and preview the data
3. Search a collection by user-provided shapefile (ESRI shp) and temporal range
4. Donwload the first file (from the PO.DAAC cloud archive to local computer) and preview subset
4. Download all data (from the PO.DAAC cloud archive to local computer) based on shp and time search criteria 

----- 

## First, import needed packages and libraries

In [ ]:
%matplotlib inline

from os.path import join, expanduser, basename
from urllib import request, parse
import requests
import shutil
import json
from http.cookiejar import CookieJar
from getpass import getpass
from netrc import netrc
import pprint
from platform import system

import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

## Earthdata Login Authentication

You'll need to create an Earthdata Login account to access data, as well as discover restricted data, from the NASA Earthdata system. [Follow these setup instructions](/get-started/earthdata-login.qmd)

The `setup_earthdata_login_auth` function will allow Python scripts to log into any Earthdata Login application programmatically.  To avoid being prompted for
credentials every time you run and also allow clients such as curl to log in, you can add the following
to a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```

Make sure that this file is only readable by the current user or you will receive an error stating
"netrc access too permissive."

`$ chmod 0600 ~/.netrc` 

In [ ]:
from netrc import netrc
from platform import system
from getpass import getpass
from http.cookiejar import CookieJar
from os.path import join, expanduser

TOKEN_DATA = ("<token>"
              "<username>%s</username>"
              "<password>%s</password>"
              "<client_id>PODAAC CMR Client</client_id>"
              "<user_ip_address>%s</user_ip_address>"
              "</token>")


def setup_cmr_token_auth(endpoint: str='cmr.earthdata.nasa.gov'):
    ip = requests.get("https://ipinfo.io/ip").text.strip()
    return requests.post(
        url="https://%s/legacy-services/rest/tokens" % endpoint,
        data=TOKEN_DATA % (input("Username: "), getpass("Password: "), ip),
        headers={'Content-Type': 'application/xml', 'Accept': 'application/json'}
    ).json()['token']['id']


def setup_earthdata_login_auth(endpoint: str='urs.earthdata.nasa.gov'):
    netrc_name = "_netrc" if system()=="Windows" else ".netrc"
    try:
        username, _, password = netrc(file=join(expanduser('~'), netrc_name)).authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        print('Please provide your Earthdata Login credentials for access.')
        print('Your info will only be passed to %s and will not be exposed in Jupyter.' % (endpoint))
        username = input('Username: ')
        password = getpass('Password: ')
    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)
    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)


# Get your authentication token for searching restricted records in the CMR:
_token = setup_cmr_token_auth(endpoint="cmr.earthdata.nasa.gov")

# Start authenticated session with URS to allow restricted data downloads:
setup_earthdata_login_auth(endpoint="urs.earthdata.nasa.gov")

-----------
### Explore data availability using the Common Metadata Repository
The Common Metadata Repository (CMR) is a robust metadata system that catalogs Earth Science data and associated service metadata records. CMR supports data search and discovery through an Application Programming Interface, or API, enabling reproducible data product and data file searches using a number of helpful variables, including geographic area, keyword, provider, and time.

General CMR API documentation: https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html

## 1. Identify a data collection of interest

Data sets are selected by data set IDs (e.g. `MODIS_T-JPL-L2P-v2019.0`). In the CMR API documentation, a data set id is referred to as a "short name". These short names are located at the top of each NSIDC data set landing page in gray above the full title: 

- go to https://search.earthdata.nasa.gov/search and type *POCLOUD* in the search box (which is the PO.DAAC data provider for cloud-based Pathfinder datasets in a restricted operational cloud environment). Click on the (i) next to the data collection of interest; the `shortname` is the subheader name below the main full data collection name.

<img align="center"
     src="EDS_info.png">
     
ShortName:
<img align="center"
     src="EDS_shortname.png">

In this tutorial we will be using the **MODIS-Terra SST L2 collection**, with the shortname: `MODIS_T-JPL-L2P-v2019.0`. Data from our collection of interest can be obtained from the PO.DAAC (Earthdata) cloud archive.

### Search by dataset shortname and provider

To search Earthdata from this notebook, we can use the following code with key words that describe our dataset (also knows as a **collection**). This snippet of Python code uses the `requests` module to get collection metadata from the CMR, with our dataset of interset *shortname* and *provider* as search criteria (parameters). In subsequet notebooks (demos), this functionality is hidden in a package called `tutorial_helper_functions` for ease of use. For now, this code snippet will help us understand how the request is made:

In [ ]:
modis_coll = requests.get(
    url="https://cmr.earthdata.nasa.gov/search/collections.umm_json",    # CMR API url
    params={'ShortName': "MODIS_T-JPL-L2P-v2019.0",      # dataset collection shortname
            'provider': "POCLOUD",      # data provider
            'token': _token},
).json()

modis_coll['items'][0]['meta']   # print collection metadata

Based on the metadata retrieved above, we now know the collection ID (*concept-id*) is `C1940475563-POCLOUD`

To **retrieve the *granule (file)* metadata** from the CMR:

In [ ]:
modis_gran = requests.get(
    url="https://cmr.earthdata.nasa.gov/search/granules.umm_json", 
    params={'ShortName': "MODIS_T-JPL-L2P-v2019.0", 
            'provider': "POCLOUD",
            'token': _token, },
).json()

modis_gran['items'][0]['meta']

In this case, the metadata tells us that the the granule ID is `G1967602341-POCLOUD`, among other information.

## 2. Download a granule & preview data

### Access & preview the data - get a quick feel for what the data looks like
Here is how to list the URLs needed to access the data:

In [ ]:
modis_gran['items'][0]['umm']['RelatedUrls']

To access POCLOUD Earthdata Cloud data, you want URLs with 'Type': `'GET DATA'` and host `https://archive.podaac.earthdata.nasa.gov`.

**Lesson note:** This URL points you to data that is archived in a cloud environment, namely, PO.DAAC's 'space' in the AWS S3 storage environment.

In [ ]:
modis_url = modis_gran['items'][0]['umm']['RelatedUrls'][1]['URL']
modis_url

### Download a file and open to explore

Download the file and open it with xarray in memory. This will pull cloud-stored data onto local space. *Note there is no charge for the user in doing so.* 

**Lesson note:** You are now downloading a file to your local space, from the PO.DAAC (Earthdata) archive that lives in the AWS cloud. Again, there is no charge for the user in doing so.

In [ ]:
with request.urlopen(modis_url) as response, open('tutorial1_data_MODIS.nc', 'wb') as out_file:
    print('Content Size:', response.headers['Content-length'])
    shutil.copyfileobj(response, out_file)
    print("Downloaded request to tutorial1_data_MODIS.nc")

You can preview metadata of the downloaded file with `ncdump`, or open in memory wtih `xarray`.

In [ ]:
#!ncdump -h tutorial1_data_MODIS.nc

In [ ]:
import xarray as xr
ds_MODIS = xr.open_dataset('tutorial1_data_MODIS.nc')
print(ds_MODIS)

Let's do a **quick plot** of the `sea_surface_temperature` variable:

In [ ]:
from pandas.plotting import register_matplotlib_converters

ds_MODIS.sea_surface_temperature.plot()

We can see the time this file represents, and the variable, but it's hard to tell where on the map we are. Let's try it another way:

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

plt.scatter(ds_MODIS.lon, ds_MODIS.lat, lw=2, c=ds_MODIS.sea_surface_temperature)
plt.colorbar()
#plt.clim(-0.3, 0.3)

plt.show()

<img align="left"
     src="MODIS_SST_1stFile.png">

## 3. Select data with shp file using the CMR API

Now that we've previewed the data and decided we'd like to request more, specific to our use case, we can pass the collection ID into the CMR API to *search* by geographic shape file. The service to *subset by shp* is also in development and should be available in 2021.

### Access data selected by geographic shapefile
We will request data overlapping the Gulf of Mexico by uploading a shape file with that boundary. This shape file can be one that you created, shred by a collaborator, or any other user, as long as it follows shape file convention. In essence, this following service allows the user to bring their own shape file to do a data search. The returned response will provide a list of data files from the cloud-based PO.DAAC archive (Earthdata Cloud) that intersect this given shape file.

This requires the use of a multipart/form-data POST request. Supported shapefile formats include ESRI, GeoJSON, and KML. The associated mime-types are as follows:

| Shapefile Format | mime-type |
|:-----------------|----------:|
| ESRI | application/shapefile+zip |
| GeoJSON | application/geo+json |
| KML | application/vnd.google-earth.kml+xml |

ESRI shapefiles must be uploaded as a single .zip file.

### Example with ESRI .shp file:

We will now search with an ESRI shapefile and a temporal bound to limit our space and time region of interest. Namely, we will search over the Gulf of Mexico during the period 1 Aug 2020 to 1 Nov 2020, since we are interested in exploring how SST responds during the latest Atlantic hurricane season. Our spatial boundary shape file to search with over the Gulf of Mexico looks like this (previewed in QGIS):

<img align="center"
     src="Gulf_Mexico_shp_QGIS_zoom.png">

**Lesson Note:** You are once again searching on data that lives in the PO.DAAC (Earthdata) cloud archive, from your local computer, by running this next code snippet. You are searching spatially (via shape file boundaries) and temporally.

(Side note: the token parameter below allows you to access this new PO.DAAC cloud operational archive space. This is possible because your Earthdata login was added to a list of early access users to this cloud environment, for this workshop.)

In [ ]:
# the URL of the CMR searvice
url = 'https://cmr.earthdata.nasa.gov/search/granules.umm_json'

#The shapefile we want to use in our search
shp_file = open('resources/gulf_shapefile.zip', 'rb')

#need to declare the file and the type we are uploading
files = {'shapefile':('gulf_shapefile.zip',shp_file, 'application/shapefile+zip')}

#used to define parameters such as the concept-id and things like temporal searches
parameters = {'collection_concept_id':'C1940475563-POCLOUD', 'token': _token, 'temporal':'2020-08-23T00:00:00Z,2020-08-29T00:00:00Z', 'page_size':33}

response = requests.post(url, files=files, params=parameters)
results  = response.json()

print(json.dumps(results['items'][0], indent=2))

## 4. Preview the selected data

The `POCLOUD` in the response above signals that you are indeed searching on data from the PO.DAAC archive within the Earthdata Cloud.

Let's take a look at the url for the first data file this spatial and temporal search result returned:

**For now, let's plot the first files** from our query, that we have downloaded to your local space.

In [ ]:
new_modis_url = results['items'][0]['umm']['RelatedUrls'][1]['URL']
new_modis_url

In [ ]:
with request.urlopen(new_modis_url) as response, open('tutorial1_data_MODIS_from_shp.nc', 'wb') as out_file:
    print('Content Size:', response.headers['Content-length'])
    shutil.copyfileobj(response, out_file)
    print("Downloaded request to tutorial1_data_MODIS_from_shp.nc")

In [ ]:
ds_MODIS_shp = xr.open_dataset('tutorial1_data_MODIS_from_shp.nc')
ds_MODIS_shp

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

plt.scatter(ds_MODIS_shp.lon, ds_MODIS_shp.lat, lw=2, c=ds_MODIS_shp.sea_surface_temperature)
plt.colorbar()
#plt.clim(-0.3, 0.3)

This file from 23 Aug 2020 intersects our shp boundary. In this case, overlap is not extensive, but it does overlap. We can confirm our search by doing the same query in the **Earthdata Search** portal mentioned at the beginning of the tutorial. https://search.earthdata.nasa.gov/search/granules?projectId=8195834861

Indeed the granule returned matches the search boundary: 

<img align="center"
     src="EDS_shp_granule1.png">

## 5. Download after spatial and temporal selection
Loop to download the MODIS SST files based on our search criteria (those that intersect the region of interest (defined by our shapefile here) and our period of interest.

In [ ]:
urls = []
for r in results['items']:
    for u in r['umm']['RelatedUrls']:
        if u['URL'].startswith("https://archive.podaac") and u['Type']=="GET DATA":
            urls.append(u['URL'])
urls

In [ ]:
for u in urls:
    print(u)
    with request.urlopen(u) as response, open(f'{basename(u)}', 'wb') as out_file:
        print('Content Size:', response.headers['Content-length'])
        shutil.copyfileobj(response, out_file)
        print(f"Downloaded request to {basename(u)}")

We now have the 33 files from our search downloaded to our local environment (computer). 

<img align="center"
     src="files_download.png">

## Tutorial Summary & Discusssion

We've searched for a dataset of interest that is archived in the PO.DAAC Earthdata Cloud, previewed it, selected certain data granules that matched a user-defined spatial area (by using .shp files) and temporal range, and downloaded the serach results to our local computer, from the Earthdata Cloud.

From here, additional analysis can be done. In future guiding notebooks, handling of MODIS L2 SST data into analysis ready data (ARD) 'data cubes'or 'time series stacks' will also be made available, which further shows how to treat level 2 data to create level 3-like uniformly gridded L2 data to your exact specifications.    

And as mentioned at the beginning of the notebook, in this example we are using a shape file to search on ocean remote sensing data, although one can do this type of search on any Earthdata (DAAC) data. For example, of particular interest could be using shape files to search on terrestrial hydrology data (e.g watersheds), globally (depending on data availability of course), or other coastal regions with unique boundaries.
